<a href="https://colab.research.google.com/github/antoronson/Vision-Based-Tracking/blob/dev/Vision_Based_Tracking/Object_Tracking_From_Video_Stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install ultralytics
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil
import imageio



In [7]:
from IPython.display import HTML

from base64 import b64encode

def play_mp4(file_path):
  mp4 = open(file_path, 'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

  # Add "return" here!
  return HTML(f"""
  <video width="640" height="480" controls>
      <source src="{data_url}" type="video/mp4">
  </video>
  """)

In [8]:
model = YOLO('yolov8n.pt')

dir = "/content/drive/MyDrive/datasets/yolo_video"

input_file = "input_file.mp4"

output_file = "output_file.mp4"

input_path = os.path.join(dir, input_file)

output_path = os.path.join(dir, output_file)

#play_mp4(input_path)

In [9]:
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print (f"Width of video: {width}")
print (f"Height of video: {height}")
print (f"FPS of video: {fps}")

frame_count = 0
frames = []

while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break

  results = model.track(frame, persist=True, classes=[2], conf=0.4)
  marked_frame = results[0].plot()
  marked_frame_rgb = cv2.cvtColor(marked_frame, cv2.COLOR_BGR2RGB)
  frames.append(marked_frame_rgb)

  #cv2.imshow("frame", marked_frame)
  #if frame_count %10 == 0:
   # cv2_imshow  (marked_frame)
  frame_count += 1
  if frame_count >= 20:
    break

  #if cv2.waitKey(1) & 0xFF == ord("q"):
   # break

cap.release()
cv2.destroyAllWindows()


# Converting to Gif
gif_file = "output.gif"
gif_path = os.path.join(dir, gif_file)
imageio.mimsave(gif_path, frames, fps=10)


Width of video: 3840
Height of video: 2160
FPS of video: 60


'\nframe_count = 0\nframes = []\n\nwhile cap.isOpened():\n  ret, frame = cap.read()\n  if not ret:\n    break\n\n  results = model.track(frame, persist=True, classes=[2], conf=0.4)\n  marked_frame = results[0].plot()\n  marked_frame_rgb = cv2.cvtColor(marked_frame, cv2.COLOR_BGR2RGB)\n  frames.append(marked_frame_rgb)\n\n  #cv2.imshow("frame", marked_frame)\n  #if frame_count %10 == 0:\n   # cv2_imshow  (marked_frame)\n  frame_count += 1\n  if frame_count >= 20:\n    break\n\n  #if cv2.waitKey(1) & 0xFF == ord("q"):\n   # break\n\ncap.release()\ncv2.destroyAllWindows()\n\n\n# Converting to Gif\ngif_file = "output.gif"\ngif_path = os.path.join(dir, gif_file)\nimageio.mimsave(gif_path, frames, fps=10)\n\n'

In [10]:
#from IPython.display import Image, display
#display(Image(filename=gif_path))


As we see the above model is not sophisticated. Though it plots the box arounf object and stores as image, the output is not how it was expected. Further the algorithm cannot detect the van again frame to frame and the busses remains undetected. Thus the algorithm has to be improved and the data presentation format should also be changed.

# 07.01.2026:
During the test the model kept crashing. When noticed, we ended up with the problem of buffer overflowing. Therefore, the image will be written back to mp4 file on the fly and then the video with the embedded frame will be played back. A second gif file with compressed frames will also be tried.

